In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import project_path

In [6]:
import glob
import os

from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from src.plotting import make_result_heatmap, add_lines

In [7]:
pd.read_csv('../experiments.csv')

,id,Type of experiment
0,0,Local experiments
1,1587919855,p vs q algorithms for SBM with 50 cuts
2,1587918440,p vs q algorithms for SBM with 50 cuts for 4 ...
3,1587804052,p vs q algorithms for SBM with 50 cuts for 5 ...
4,1587703699,KnnBlobs experiments with different k and 50 ...
5,1586946592,SBM experiments
6,1587621343,KnnBlobs with 100 cuts testing different a
7,1587629564,KL vs MF algorithms on SBM
8,1587632158,SBM with fixed p and q and different numbers ...
9,1587632375,KnnBlobs with fixed k and different numbers o...


In [8]:
experiment_name = '1589802894'

In [9]:
path_in = Path(f'../output/{experiment_name}').resolve()
path_out = Path(f'../plots/{experiment_name}').resolve()
path_out.mkdir(parents=True, exist_ok=True)

In [10]:
full_df = pd.DataFrame()
for subdir, dirs, files in os.walk(path_in):
    for file in files:
        
        current_df = pd.read_csv(f'{subdir}/{file}', index_col=0)
        full_df = full_df.append(current_df)
full_df = full_df.reset_index(drop=True)

In [11]:
full_df

,agreement,ars,block_sizes,dataset_name,dataset_type,lb_f,max_order,nb_cuts,order_best,p,percentile_orders,preprocessing_name,q,seed,unique_id
0,50.0,0.000000,"[100, 100]",sbm,graph,0.2,5160.0,50.0,3315.0,0.28,100.0,fid_mat,0.83,42.0,1589802894
1,50.0,0.000000,"[100, 100]",sbm,graph,0.3,7616.0,50.0,6087.0,0.60,100.0,fid_mat,0.92,42.0,1589802894
2,50.0,0.000000,"[100, 100]",sbm,graph,0.4,5311.0,50.0,4722.0,0.18,100.0,fid_mat,0.87,42.0,1589802894
3,50.0,0.000000,"[100, 100]",sbm,graph,0.4,4887.0,50.0,4252.0,0.28,100.0,fid_mat,0.69,42.0,1589802894
4,50.0,0.590879,"[100, 100]",sbm,graph,0.4,7805.0,50.0,7604.0,0.87,100.0,fid_mat,0.78,42.0,1589802894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,50.0,0.459964,"[100, 100]",sbm,graph,0.3,4590.0,50.0,3542.0,0.55,100.0,fid_mat,0.37,42.0,1589802894
11996,50.0,1.000000,"[100, 100]",sbm,graph,0.4,512.0,50.0,512.0,0.55,100.0,fid_mat,0.05,42.0,1589802894
11997,50.0,1.000000,"[100, 100]",sbm,graph,0.3,5292.0,50.0,3217.0,0.74,100.0,fid_mat,0.32,42.0,1589802894
11998,50.0,0.000000,"[100, 100]",sbm,graph,0.2,824.0,50.0,531.0,0.09,100.0,fid_mat,0.14,42.0,1589802894


# SBM

### Plot SBM for 2 blocks and different a id = 1587639002

In [23]:
if experiment_name == '1587639002':
        
    full_df = full_df[full_df['preprocessing_name'] == 'fid_mat']
    experiments = [x for _, x in full_df.groupby(['block_sizes', 'lb_f'])]
    for experiment in experiments:
        
        fig, ax = plt.subplots(figsize=(15, 10))

        title = f"SBM with {experiment['block_sizes'].iloc[0]} blocks using p = 0.3, nb_cuts = 100, lb_f = {experiment['lb_f'].iloc[0]}"
        name = f"block_sizes_{experiment['block_sizes'].iloc[0]}_lb_f_{experiment['lb_f'].iloc[0]}"
        
        columns = ['q', 'agreement', 'homogeneity']
        data = experiment[columns]
                                                 
        make_result_heatmap(data, title, ax, x_column=columns[0], y_column=columns[1], values_column=columns[2])

        fig.tight_layout()

        fig.savefig(f'{path_out / name}.svg')
        plt.close(fig) 

### Plot SBM for 2 blocks with theoretical bounds id = 1589802894

In [12]:
block_size = 100
xs = np.arange(1, 21)
i = (np.log(block_size) / block_size * np.arange(1, 21)).round(2)

idx = np.arange(len(xs))
mask = np.zeros((len(xs), len(xs)), dtype=bool)

for a, b in product(xs, xs):
    if a >= b:
        mask[a-1, b-1] = (np.abs(np.sqrt(a) - np.sqrt(b)) >= np.sqrt(2))
theory_df = pd.DataFrame(mask, columns=i, index=i).T.sort_index(ascending=False).sort_index(axis=1, ascending=True)
values_theory = theory_df.to_numpy()

In [21]:
if experiment_name == '1589802894':
    experiments = [x for _, x in full_df.groupby(['lb_f'])]
    for experiment in experiments:

        fig, ax = plt.subplots(figsize=(15, 10))

        name = f"SBM_theory_lb_f_{experiment['lb_f'].iloc[0]}"

        columns = ['p', 'q', 'ars']
        data = experiment[columns].groupby(['p', 'q'], as_index=False).mean()
                                                 
        make_result_heatmap(data, ax, x_column=columns[0], y_column=columns[1], values_column=columns[2])
        add_lines(values_theory, ax)
        ax.set_axisbelow(True)
        fig.tight_layout()

        fig.savefig(f'{path_out / name}.pdf')
        plt.close(fig) 

### Plot SBM for 2/3 blocks and unbalanced id = 1587919855

In [76]:
if experiment_name == '1587919855':
        
    full_df = full_df[full_df['preprocessing_name'] == 'fid_mat']
    experiments = [x for _, x in full_df.groupby(['block_sizes', 'lb_f', 'agreement'])]
    for experiment in experiments:
        
        fig, ax = plt.subplots(figsize=(16.18, 10))

        title = f"SBM with {experiment['block_sizes'].iloc[0]} blocks using nb_cuts = 50, lb_f = {experiment['lb_f'].iloc[0]} and a = {experiment['agreement'].astype(int).iloc[0]}"
        name = f"block_sizes_{experiment['block_sizes'].iloc[0]}_a_{experiment['agreement'].iloc[0]}_lb_f_{experiment['lb_f'].iloc[0]}"
        
        data = experiment[['p', 'q', 'homogeneity']]
        make_result_heatmap(data, title, ax)

        fig.tight_layout()

        fig.savefig(f'{path_out / name}.pdf')
        plt.close(fig) 

### Plot SBM for 4 blocks id = 1587918440

In [27]:
if experiment_name == '1587918440':
        
    full_df = full_df[full_df['preprocessing_name'] == 'fid_mat']
    experiments = [x for _, x in full_df.groupby(['block_sizes', 'lb_f', 'agreement'])]
    for experiment in experiments:
        
        fig, ax = plt.subplots(figsize=(15, 10))

        title = f"SBM with {experiment['block_sizes'].iloc[0]} blocks using nb_cuts = 50, lb_f = {experiment['lb_f'].iloc[0]} and a = {experiment['agreement'].astype(int).iloc[0]}"
        name = f"block_sizes_{experiment['block_sizes'].iloc[0]}_a_{experiment['agreement'].iloc[0]}_lb_f_{experiment['lb_f'].iloc[0]}"
        
        data = experiment[['p', 'q', 'homogeneity']]
        make_result_heatmap(data, title, ax)

        fig.tight_layout()

        fig.savefig(f'{path_out / name}.svg')
        plt.close(fig) 